In [1]:
import psycopg2
from datetime import timedelta
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="mimic",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()

cur.execute("SELECT version();")
print(cur.fetchone())

('PostgreSQL 15.2, compiled by Visual C++ build 1914, 64-bit',)


In [61]:
# Connect to db
conn = psycopg2.connect(host='localhost', dbname='mimic', user='postgres', password='postgres', options='-c search_path=mimiciii')
#conn = psycopg2.connect(dbname='mimic', user='postgres')
cur = conn.cursor() 

# Read in table with patients & admissions (inner join on subject_id) and icu_stays (inner joinon subject_id and hadm_id)
icustay_details = pd.read_sql_query("SELECT * FROM mimiciii.flicu_icustay_detail;", conn)

# Read in vital and lab signs
pivoted_vital = pd.read_sql_query("SELECT * FROM mimiciii.pivoted_vital;", conn)
pivoted_lab = pd.read_sql_query("SELECT * FROM mimiciii.ckd_pivoted_lab;", conn)

# Read in lab measurements
# Use flicu_pivoted_lab (as it only takes the lab tests during ICU stay)
#query = "SELECT * FROM mimiciii.flicu_pivoted_lab;"
# Alternative:  Use the lab values recorded previous to the ICU stay (although during same hospital admission!), 
# then sample them  (8h intervalls) and then forward fill plus cap at either icu admission time of first vital sign recorded
#query = "SELECT * FROM mimiciii.pivoted_lab;"
#pivoted_lab = pd.read_sql_query(query, conn)

# Close the cursor and connection to so the server can allocate bandwidth to other requests
cur.close()
conn.close()

C:\Users\aakru\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\aakru\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\aakru\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [59]:
icustay_details.describe()

,subject_id,hadm_id,icustay_id,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,label_death_icu,label_cor_art,diabetes_mellitus,ckd,anemia_flag
count,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000,17188.000000
mean,30961.836805,149698.800442,249870.240168,20.709839,61.125928,0.163893,1.380556,13.277353,1.097161,0.119851,0.180242,0.169421,0.081394,0.133116
std,26970.598264,28916.126126,28813.894228,19.563438,53.308340,0.370190,1.136666,15.241442,0.366591,0.324797,0.384400,0.375134,0.273447,0.339710
min,3.000000,100001.000000,200003.000000,0.264583,0.000007,0.000000,1.000000,4.000046,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10923.750000,124896.500000,224968.500000,9.229167,41.608982,0.000000,1.000000,5.297300,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,22132.500000,149350.000000,249943.500000,14.655208,62.169065,0.000000,1.000000,7.985093,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,46219.750000,174736.750000,274755.750000,24.556597,75.964002,0.000000,1.000000,14.469771,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,99985.000000,199994.000000,299994.000000,294.660417,310.280861,1.000000,28.000000,173.072512,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Setting window length 

In [62]:
WINDOW_LENGTH = 24*4

Keeping records that are atleast window length

In [64]:
data= icustay_details.copy()
data = data[data.los_icu >= WINDOW_LENGTH/24.0]

In [65]:
filtered_icustay_ids = pd.DataFrame(data['icustay_id'].unique(), columns=['icustay_id'])

In [66]:
# Drop measurements with no belonging icustay_id
pivoted_vital = pivoted_vital.dropna(subset=['icustay_id'])
pivoted_lab = pivoted_lab.dropna(subset=['icustay_id'])

# Cast icustay_id types to int
pivoted_vital['icustay_id'] = pivoted_vital['icustay_id'].astype(int)
pivoted_lab['icustay_id'] = pivoted_lab['icustay_id'].astype(int)

# Keep only values of patients in previously filtered icustay_ids in labs and vitals
pivoted_vital = pivoted_vital.merge(filtered_icustay_ids, on='icustay_id', how='right').drop_duplicates()
pivoted_lab = pivoted_lab.merge(filtered_icustay_ids, on='icustay_id', how='right').drop_duplicates()

In [67]:
# Min of each lab and vitals
icustay_ids_charttime_min_lab = pivoted_lab[["icustay_id", "charttime"]][pivoted_lab.groupby("icustay_id")["charttime"].rank(ascending=1,method='dense') == 1]
icustay_ids_charttime_min_vital = pivoted_vital[["icustay_id", "charttime"]][pivoted_vital.groupby("icustay_id")["charttime"].rank(ascending=1,method='dense') == 1]
# Min of both combined
icustay_ids_charttime_min_vital_lab = pd.concat([icustay_ids_charttime_min_lab, icustay_ids_charttime_min_vital], ignore_index=True)
icustay_ids_charttime_min_vital_lab = icustay_ids_charttime_min_vital_lab[["icustay_id", "charttime"]][icustay_ids_charttime_min_vital_lab.groupby("icustay_id")["charttime"].rank(ascending=1,method='dense') == 1]

# Max of each lab and vitals
icustay_ids_charttime_max_lab = pivoted_lab[["icustay_id", "charttime"]][pivoted_lab.groupby("icustay_id")["charttime"].rank(ascending=0,method='dense') == 1]
icustay_ids_charttime_max_vital = pivoted_vital[["icustay_id", "charttime"]][pivoted_vital.groupby("icustay_id")["charttime"].rank(ascending=0,method='dense') == 1]
# Max of both combined
icustay_ids_charttime_max_vital_lab = pd.concat([icustay_ids_charttime_max_lab, icustay_ids_charttime_max_vital], ignore_index=True)
icustay_ids_charttime_max_vital_lab = icustay_ids_charttime_max_vital_lab[["icustay_id", "charttime"]][icustay_ids_charttime_max_vital_lab.groupby("icustay_id")["charttime"].rank(ascending=0,method='dense') == 1]


In [78]:
# Find for which icustay_ids there exist at least WINDOW_LENGTH of data
icustay_ids_vital_lab_charttime_min_max = pd.concat([icustay_ids_charttime_max_vital_lab, icustay_ids_charttime_min_vital_lab], ignore_index=True)
time_window = timedelta(days=4, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=WINDOW_LENGTH, weeks=0)
is_time_diff_bigger_window_lab = icustay_ids_vital_lab_charttime_min_max.groupby(['icustay_id'])['charttime'].transform(lambda x: (x.max()-x.min())) >= time_window

icustay_ids_vital_lab_charttime_min_max_filtered = icustay_ids_vital_lab_charttime_min_max[is_time_diff_bigger_window_lab]
print("Unique icu stays in icustay_ids_vital_lab_charttime_min_max_filtered after filtering", icustay_ids_vital_lab_charttime_min_max_filtered['icustay_id'].nunique())

# Keep only icustay ids for which at least WINDOW_LENGTH of data exists
icustay_ids_time_filtered = pd.DataFrame(icustay_ids_vital_lab_charttime_min_max_filtered['icustay_id'].unique(), columns=['icustay_id'])
print("Unique icu stays in icustay_ids_time_filtered: ", icustay_ids_time_filtered['icustay_id'].nunique())

Unique icu stays in icustay_ids_vital_lab_charttime_min_max_filtered after filtering 16752
Unique icu stays in icustay_ids_time_filtered:  16752


In [79]:
filtered_icustay_ids = filtered_icustay_ids.merge(icustay_ids_time_filtered, on='icustay_id', how='inner').drop_duplicates()

In [80]:
demographics_filtered = data.merge(filtered_icustay_ids, on='icustay_id', how='right').drop_duplicates()
print("Number of ICU stays demographics: ", demographics_filtered['icustay_id'].nunique())

vital_filtered = pivoted_vital.merge(filtered_icustay_ids, on='icustay_id', how='right').drop_duplicates()
print("Number of ICU stays vitals: ", vital_filtered['icustay_id'].nunique())

lab_filtered = pivoted_lab.merge(filtered_icustay_ids, on='icustay_id', how='right').drop_duplicates()
print("Number of ICU stays labs: ", lab_filtered['icustay_id'].nunique())

Number of ICU stays demographics:  16752
Number of ICU stays vitals:  16752
Number of ICU stays labs:  16752


In [76]:
lab_filtered['icustay_id'].unique()

array([228400, 299609, 218958, ..., 241017, 270667, 224889], dtype=int64)

In [81]:
lab_filtered[lab_filtered['icustay_id']==228400]['charttime'].max() - lab_filtered[lab_filtered['icustay_id']==228400]['charttime'].min()

Timedelta('5 days 16:11:00')

In [82]:
lab_filtered[lab_filtered['icustay_id']==299609]['charttime'].max() - lab_filtered[lab_filtered['icustay_id']==299609]['charttime'].min()

Timedelta('3 days 07:59:00')

In [83]:
vital_filtered[vital_filtered['icustay_id']==228400]['charttime'].max() - vital_filtered[vital_filtered['icustay_id']==228400]['charttime'].min()

Timedelta('6 days 02:28:00')

In [84]:
demographics_filtered[demographics_filtered['icustay_id']==228400]['los_icu']

0    6.285637
Name: los_icu, dtype: float64

In [85]:
vital_filtered = vital_filtered.merge(lab_filtered[['icustay_id', 'charttime']], on=['icustay_id', 'charttime'], how='outer').drop_duplicates()
print("Number of ICU stays in lab_filtered: ", vital_filtered['icustay_id'].nunique())
lab_filtered = lab_filtered.merge(vital_filtered[['icustay_id', 'charttime']], on=['icustay_id', 'charttime'], how='outer').drop_duplicates()
print("Number of ICU stays in lab_filtered: ", lab_filtered['icustay_id'].nunique())

Number of ICU stays in lab_filtered:  16752
Number of ICU stays in lab_filtered:  16752


In [86]:
lab_filtered[lab_filtered['icustay_id']==299609]['charttime'].max() - lab_filtered[lab_filtered['icustay_id']==299609]['charttime'].min()

Timedelta('4 days 00:00:00')

In [88]:
vital_filtered[vital_filtered['icustay_id']==299609]['charttime'].max() - vital_filtered[vital_filtered['icustay_id']==299609]['charttime'].min()

Timedelta('4 days 00:00:00')

In [90]:
lab_filtered[lab_filtered['icustay_id']==228400]['charttime'].max() - lab_filtered[lab_filtered['icustay_id']==228400]['charttime'].min()

Timedelta('6 days 02:28:00')

Now keep only uptil 4 days data

In [105]:
delta_t_data = timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=WINDOW_LENGTH, weeks=0)
demographics_windowed = demographics_filtered.copy()
demographics_windowed['predtime'] = demographics_windowed.intime + delta_t_data
demographics_windowed['delta_t_pred'] = demographics_windowed.outtime - demographics_windowed.predtime

demographics_windowed[['subject_id', 'icustay_id', 'intime', 'predtime', 'delta_t_pred']].head(5)

,subject_id,icustay_id,intime,predtime,delta_t_pred
0,58817,228400,2118-05-02 06:24:29,2118-05-06 06:24:29,2 days 06:51:19
1,23553,299609,2118-12-24 20:14:43,2118-12-28 20:14:43,0 days 00:19:33
2,14137,218958,2132-08-24 17:07:00,2132-08-28 17:07:00,10 days 00:34:00
3,28970,241427,2151-11-07 01:49:18,2151-11-11 01:49:18,27 days 17:57:20
4,2269,272085,2187-12-04 11:35:04,2187-12-08 11:35:04,12 days 02:08:02


In [103]:
cut_icustay_ids = pd.DataFrame(demographics_windowed['icustay_id'].unique(), columns=['icustay_id'])
print("Number of ICU stays: ", cut_icustay_ids['icustay_id'].count())

vitals_cut = vital_filtered.merge(cut_icustay_ids, on='icustay_id', how='right')
print("Number of ICU stays in vitals_cut: ", vitals_cut['icustay_id'].nunique())

labs_cut = lab_filtered.merge(cut_icustay_ids, on='icustay_id', how='right')
print("Number of ICU stays in labs_cut: ", labs_cut['icustay_id'].nunique())


Number of ICU stays:  16752
Number of ICU stays in vitals_cut:  16752
Number of ICU stays in labs_cut:  16752


In [97]:
print(delta_t_data)

4 days, 0:00:00


In [106]:
vitals_windowed = vital_filtered.merge(demographics_windowed[['icustay_id', 'predtime', 'delta_t_pred']], on='icustay_id', how='right')
vitals_windowed = vitals_windowed[vitals_windowed.charttime < vitals_windowed.predtime]
print("Number of ICU stays in vitals_windowed: ", vitals_windowed['icustay_id'].nunique())

labs_windowed = lab_filtered.merge(demographics_windowed[['icustay_id', 'predtime', 'delta_t_pred']], on='icustay_id', how='right')
labs_windowed = labs_windowed[labs_windowed.charttime < labs_windowed.predtime]
print("Number of ICU stays in labs_windowed: ", labs_windowed['icustay_id'].nunique())

windowed_icustay_ids = pd.DataFrame(pd.concat([vitals_windowed['icustay_id'], labs_windowed['icustay_id']]).unique(), columns=['icustay_id'])
demographics_windowed = demographics_windowed.merge(windowed_icustay_ids, on='icustay_id', how='right')
print("Number of ICU stays: ", demographics_windowed['icustay_id'].nunique())
print("Number of ICU deaths: ", demographics_windowed['label_death_icu'].value_counts()[1])

Number of ICU stays in vitals_windowed:  16747
Number of ICU stays in labs_windowed:  16747
Number of ICU stays:  16747
Number of ICU deaths:  1997


In [109]:
labs_windowed

,icustay_id,subject_id,charttime,aniongap,albumin,bands,bicarbonate,bilirubin,creatinine,chloride,...,ptt,inr,pt,sodium,bun,wbc,bacteria,ckd,predtime,delta_t_pred
1,228400,58817.0,2118-05-02 10:12:00,13.0,4.1,NaN,23.0,0.7,0.9,108.0,...,19.1,1.0,12.1,140.0,11.0,11.6,NaN,0.0,2118-05-06 06:24:29,2 days 06:51:19
2,228400,58817.0,2118-05-03 02:25:00,13.0,NaN,NaN,22.0,NaN,0.9,106.0,...,NaN,NaN,NaN,138.0,14.0,NaN,NaN,0.0,2118-05-06 06:24:29,2 days 06:51:19
3,228400,58817.0,2118-05-04 00:43:00,12.0,3.6,NaN,20.0,NaN,0.8,110.0,...,19.8,1.1,12.7,139.0,18.0,9.3,NaN,0.0,2118-05-06 06:24:29,2 days 06:51:19
4,228400,58817.0,2118-05-05 02:30:00,11.0,3.6,NaN,19.0,NaN,0.8,109.0,...,20.4,1.0,11.7,136.0,18.0,6.4,NaN,0.0,2118-05-06 06:24:29,2 days 06:51:19
5,228400,58817.0,2118-05-06 02:55:00,14.0,3.5,NaN,19.0,NaN,0.8,113.0,...,20.8,1.0,12.1,142.0,21.0,8.7,NaN,0.0,2118-05-06 06:24:29,2 days 06:51:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6920045,224889,NaN,2114-03-15 21:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2114-03-16 01:26:12,19 days 16:15:36
6920046,224889,NaN,2114-03-15 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2114-03-16 01:26:12,19 days 16:15:36
6920047,224889,NaN,2114-03-15 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2114-03-16 01:26:12,19 days 16:15:36
6920048,224889,NaN,2114-03-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2114-03-16 01:26:12,19 days 16:15:36


In [108]:
labs_windowed[labs_windowed['icustay_id']==228400]['charttime'].max() - labs_windowed[labs_windowed['icustay_id']==228400]['charttime'].min()

Timedelta('3 days 20:28:00')

In [110]:
count = 0
for icustay_id in labs_windowed['icustay_id'].unique():
    diff = labs_windowed[labs_windowed['icustay_id']==icustay_id]['charttime'].max() - labs_windowed[labs_windowed['icustay_id']==icustay_id]['charttime'].min()
    if diff < pd.Timedelta(days=4):
        count += 1
    print(diff)
print(count)

3 days 20:28:00
4 days 00:00:00
4 days 00:55:00
4 days 00:00:00
3 days 23:30:00
4 days 00:30:00
3 days 23:00:00
4 days 00:32:00
4 days 00:45:00
3 days 21:39:00
4 days 00:56:00
4 days 00:15:00
3 days 23:10:00
4 days 00:00:00
3 days 21:45:00
3 days 19:59:00
4 days 00:00:00
3 days 04:25:00
3 days 23:30:00
4 days 00:00:00
3 days 23:00:00
3 days 23:07:00
3 days 23:01:00
3 days 23:21:00
3 days 22:23:00
3 days 22:30:00
3 days 23:49:00
3 days 23:00:00
3 days 21:42:00
4 days 01:00:00
3 days 22:47:00
3 days 22:10:00
3 days 23:33:00
4 days 00:47:00
3 days 21:00:00
4 days 00:00:00
3 days 23:39:00
4 days 00:15:00
3 days 22:59:00
4 days 02:01:00
4 days 01:00:00
3 days 23:08:00
4 days 00:00:00
3 days 22:20:00
4 days 02:50:00
4 days 00:30:00
3 days 21:00:00
4 days 01:00:00
3 days 23:20:00
3 days 23:56:00
3 days 23:30:00
3 days 23:15:00
4 days 00:30:00
3 days 23:50:00
3 days 22:52:00
3 days 20:42:00
4 days 10:00:00
3 days 23:10:00
3 days 23:28:00
3 days 23:10:00
3 days 23:13:00
3 days 22:00:00
3 days 2

3 days 23:30:00
3 days 23:49:00
3 days 23:30:00
4 days 00:00:00
4 days 00:25:00
3 days 23:21:00
4 days 00:00:00
3 days 23:56:00
4 days 00:30:00
4 days 00:45:00
3 days 20:28:00
3 days 23:30:00
4 days 00:50:00
3 days 23:00:00
3 days 23:16:00
3 days 22:51:00
4 days 00:00:00
4 days 00:45:00
3 days 22:15:00
3 days 22:33:00
4 days 00:00:00
3 days 22:27:00
3 days 23:16:00
4 days 00:15:00
4 days 00:00:00
3 days 23:25:00
4 days 02:47:00
4 days 00:35:00
4 days 00:45:00
3 days 23:00:00
3 days 22:40:00
3 days 23:49:00
3 days 22:55:00
3 days 23:35:00
3 days 23:45:00
3 days 21:42:00
3 days 23:00:00
4 days 00:00:00
3 days 22:30:00
4 days 00:00:00
4 days 08:17:00
4 days 00:24:00
3 days 23:00:00
3 days 23:00:00
3 days 23:35:00
3 days 22:11:00
3 days 23:09:00
4 days 00:11:00
3 days 23:00:00
4 days 00:45:00
3 days 23:30:00
3 days 23:05:00
4 days 00:00:00
3 days 23:06:00
3 days 23:07:00
4 days 01:50:00
3 days 22:30:00
3 days 23:30:00
4 days 00:33:00
3 days 23:41:00
3 days 23:31:00
4 days 00:00:00
4 days 0

3 days 20:00:00
3 days 23:15:00
3 days 23:30:00
3 days 23:30:00
4 days 00:00:00
3 days 23:00:00
4 days 00:15:00
3 days 23:45:00
4 days 00:00:00
4 days 00:18:00
3 days 23:00:00
3 days 21:56:00
3 days 23:12:00
4 days 00:00:00
3 days 23:45:00
4 days 00:30:00
3 days 23:07:00
3 days 23:08:00
3 days 23:35:00
3 days 22:59:00
3 days 22:03:00
3 days 22:55:00
4 days 00:00:00
3 days 22:32:00
3 days 23:33:00
3 days 21:00:00
3 days 22:35:00
3 days 22:15:00
3 days 22:50:00
3 days 23:20:00
3 days 23:45:00
3 days 23:09:00
4 days 09:00:00
4 days 00:00:00
4 days 00:00:00
4 days 01:00:00
4 days 01:00:00
3 days 22:45:00
3 days 22:37:00
3 days 23:45:00
3 days 23:39:00
3 days 22:24:00
4 days 00:00:00
3 days 23:00:00
4 days 00:00:00
3 days 23:30:00
3 days 23:30:00
4 days 00:15:00
3 days 23:10:00
3 days 23:15:00
4 days 01:06:00
4 days 00:40:00
3 days 23:00:00
4 days 00:04:00
4 days 01:00:00
3 days 23:20:00
3 days 22:00:00
3 days 23:15:00
3 days 23:00:00
3 days 22:15:00
4 days 04:47:00
4 days 05:00:00
4 days 0

4 days 00:30:00
4 days 00:06:00
3 days 23:27:00
4 days 00:00:00
3 days 22:31:00
3 days 23:20:00
3 days 23:30:00
4 days 00:29:00
4 days 00:04:00
4 days 00:30:00
3 days 23:29:00
3 days 23:30:00
4 days 00:00:00
3 days 23:55:00
3 days 23:14:00
3 days 22:30:00
3 days 23:33:00
3 days 23:00:00
4 days 01:00:00
3 days 21:15:00
3 days 22:37:00
3 days 22:40:00
3 days 22:30:00
4 days 01:54:00
3 days 22:00:00
3 days 21:40:00
4 days 01:45:00
4 days 00:45:00
3 days 23:00:00
3 days 23:11:00
3 days 23:44:00
3 days 23:00:00
3 days 23:53:00
3 days 22:30:00
4 days 00:15:00
4 days 00:05:00
3 days 23:00:00
3 days 21:57:00
3 days 22:10:00
3 days 23:38:00
3 days 22:00:00
3 days 23:45:00
4 days 00:00:00
4 days 00:15:00
3 days 19:30:00
3 days 23:50:00
3 days 23:27:00
3 days 22:00:00
4 days 00:30:00
3 days 22:00:00
3 days 23:45:00
4 days 00:00:00
4 days 00:00:00
4 days 01:00:00
3 days 23:00:00
3 days 23:30:00
3 days 23:10:00
4 days 00:00:00
3 days 23:57:00
3 days 23:50:00
3 days 22:06:00
4 days 00:30:00
3 days 2

3 days 22:00:00
4 days 00:22:00
3 days 22:01:00
3 days 23:22:00
4 days 00:30:00
3 days 23:16:00
4 days 00:30:00
4 days 00:15:00
3 days 22:26:00
3 days 23:00:00
4 days 01:00:00
3 days 23:00:00
3 days 23:36:00
3 days 23:27:00
3 days 23:15:00
4 days 00:00:00
3 days 21:59:00
3 days 22:42:00
4 days 00:00:00
4 days 00:00:00
4 days 00:30:00
4 days 00:10:00
3 days 23:01:00
4 days 01:00:00
4 days 00:50:00
3 days 23:40:00
3 days 23:05:00
4 days 01:00:00
3 days 23:45:00
3 days 23:38:00
3 days 19:05:00
3 days 23:05:00
3 days 22:17:00
3 days 23:16:00
4 days 00:06:00
4 days 00:00:00
3 days 23:45:00
4 days 00:00:00
3 days 23:32:00
3 days 23:20:00
3 days 23:45:00
4 days 00:20:00
4 days 00:00:00
4 days 03:52:00
3 days 22:30:00
3 days 22:38:00
4 days 10:31:00
4 days 00:52:00
4 days 00:00:00
4 days 00:31:00
3 days 23:40:00
3 days 23:14:00
3 days 23:00:00
4 days 00:15:00
4 days 00:00:00
3 days 21:28:00
3 days 22:43:00
4 days 01:58:00
3 days 20:28:00
3 days 23:15:00
4 days 00:00:00
4 days 00:05:00
4 days 0

3 days 23:00:00
3 days 23:30:00
4 days 00:50:00
3 days 23:17:00
4 days 02:00:00
3 days 22:07:00
3 days 23:29:00
3 days 23:30:00
4 days 00:15:00
4 days 01:00:00
3 days 22:17:00
4 days 00:00:00
4 days 00:30:00
4 days 00:06:00
3 days 22:00:00
4 days 00:00:00
4 days 00:00:00
3 days 22:30:00
3 days 23:29:00
3 days 22:30:00
3 days 17:00:00
4 days 00:00:00
4 days 01:00:00
3 days 22:46:00
4 days 00:30:00
4 days 00:00:00
3 days 23:02:00
3 days 23:30:00
4 days 00:20:00
3 days 23:55:00
3 days 23:07:00
4 days 00:00:00
4 days 00:40:00
3 days 22:30:00
3 days 23:05:00
4 days 01:00:00
4 days 00:10:00
4 days 00:00:00
4 days 00:34:00
3 days 23:15:00
3 days 23:00:00
3 days 23:40:00
3 days 23:07:00
4 days 00:00:00
3 days 23:45:00
3 days 23:00:00
3 days 21:30:00
4 days 02:53:00
3 days 23:35:00
3 days 22:24:00
3 days 21:22:00
3 days 19:33:00
4 days 00:00:00
4 days 00:28:00
4 days 00:00:00
3 days 23:15:00
3 days 22:00:00
3 days 23:29:00
4 days 00:30:00
3 days 22:59:00
3 days 23:31:00
3 days 23:25:00
3 days 1

4 days 00:00:00
4 days 00:30:00
3 days 22:20:00
3 days 23:40:00
3 days 22:00:00
4 days 00:08:00
4 days 00:30:00
4 days 01:00:00
4 days 01:10:00
4 days 00:00:00
3 days 23:45:00
3 days 22:26:00
4 days 00:15:00
3 days 20:40:00
3 days 23:53:00
4 days 01:00:00
3 days 23:02:00
3 days 22:40:00
3 days 23:30:00
3 days 23:56:00
3 days 23:15:00
3 days 23:34:00
4 days 00:00:00
4 days 00:24:00
3 days 23:00:00
4 days 00:00:00
4 days 01:05:00
4 days 00:00:00
3 days 22:48:00
3 days 23:05:00
3 days 23:10:00
3 days 21:45:00
3 days 23:00:00
4 days 01:15:00
4 days 00:00:00
3 days 21:30:00
4 days 00:30:00
3 days 22:36:00
3 days 21:00:00
3 days 21:50:00
4 days 00:30:00
3 days 23:27:00
4 days 00:45:00
3 days 23:00:00
4 days 01:00:00
3 days 23:30:00
3 days 22:49:00
3 days 23:53:00
4 days 00:00:00
3 days 23:28:00
4 days 02:50:00
3 days 23:50:00
3 days 21:17:00
3 days 23:45:00
3 days 22:36:00
3 days 21:00:00
3 days 20:14:00
3 days 23:52:00
3 days 21:00:00
4 days 00:05:00
3 days 23:30:00
4 days 00:10:00
3 days 2

4 days 00:34:00
3 days 23:15:00
3 days 23:50:00
3 days 21:17:00
3 days 23:07:00
4 days 00:16:00
4 days 00:00:00
4 days 02:45:00
3 days 17:05:00
3 days 22:00:00
3 days 21:30:00
3 days 21:15:00
4 days 00:00:00
3 days 23:49:00
3 days 23:06:00
4 days 00:00:00
3 days 23:15:00
3 days 23:15:00
4 days 00:00:00
3 days 23:00:00
4 days 00:30:00
3 days 20:10:00
4 days 00:00:00
3 days 21:00:00
3 days 22:35:00
3 days 23:01:00
3 days 23:45:00
4 days 00:45:00
4 days 00:29:00
3 days 23:00:00
3 days 23:21:00
3 days 21:54:00
4 days 01:06:00
3 days 23:45:00
3 days 23:30:00
3 days 23:47:00
3 days 23:55:00
4 days 08:00:00
4 days 00:00:00
4 days 00:00:00
4 days 00:34:00
3 days 23:00:00
3 days 21:50:00
3 days 23:00:00
4 days 00:00:00
4 days 00:30:00
3 days 23:30:00
4 days 00:00:00
3 days 23:11:00
3 days 12:45:00
3 days 23:15:00
4 days 00:30:00
4 days 00:00:00
3 days 22:30:00
4 days 06:00:00
3 days 23:43:00
3 days 23:00:00
3 days 22:00:00
4 days 00:00:00
3 days 23:00:00
4 days 00:00:00
4 days 00:00:00
3 days 2

4 days 01:59:00
4 days 00:00:00
3 days 23:42:00
3 days 22:50:00
3 days 23:27:00
4 days 00:30:00
3 days 23:50:00
3 days 23:45:00
4 days 11:00:00
3 days 23:45:00
3 days 21:57:00
3 days 19:00:00
4 days 00:01:00
4 days 00:50:00
3 days 23:00:00
3 days 22:29:00
3 days 23:29:00
3 days 23:45:00
3 days 23:30:00
4 days 00:00:00
3 days 23:00:00
4 days 01:00:00
3 days 23:05:00
4 days 00:00:00
4 days 00:40:00
4 days 00:20:00
3 days 22:30:00
3 days 21:00:00
4 days 00:00:00
4 days 01:00:00
3 days 23:00:00
4 days 00:00:00
4 days 00:30:00
3 days 22:30:00
4 days 00:00:00
4 days 01:00:00
4 days 00:15:00
3 days 23:11:00
3 days 23:46:00
3 days 23:53:00
3 days 23:30:00
3 days 23:35:00
3 days 23:45:00
3 days 23:08:00
3 days 23:41:00
3 days 22:48:00
4 days 05:25:00
4 days 00:20:00
4 days 00:00:00
4 days 00:00:00
3 days 23:17:00
3 days 23:45:00
3 days 18:30:00
3 days 23:14:00
3 days 23:16:00
4 days 00:59:00
3 days 23:33:00
4 days 00:30:00
3 days 19:46:00
4 days 00:00:00
3 days 23:26:00
4 days 00:10:00
4 days 0

KeyboardInterrupt: 